<a href="https://colab.research.google.com/github/vrra/FGAN-Build-a-thon/blob/main/Notebooks2023/PDF_parsing_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pypdf
!pip install langchain
!pip install pdfminer.six


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.6 MB/s eta 0:00:00


In [42]:
from langchain.document_loaders import PDFMinerLoader
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
loader = PDFMinerPDFasHTMLLoader("/content/sample_data/T-REC-Y.Sup71-usecases.pdf")
data = loader.load()[0]

In [5]:
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=25c038b7ba4436f925a4c043c6f8b13f4109b929f3b85a827b66071fc697bbef
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.page_content,'html.parser')
content = soup.find_all('div')

In [47]:
content[5]

<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:487px; top:314px; width:75px; height:13px;"><span style="font-family: ArialMT; font-size:13px">(07/2022)    
<br/></span></div>

In [36]:
import re
cur_fs = None
cur_text = ''
snippets = []
start_trigger_1=0
start_trigger_2=0
stop_trigger_1=0
stop_trigger_2=0
for c in content:
    print(c.get_text())
    if ('Use case id' in c.get_text()):
      start_trigger_1=1
      start_trigger_2=0
      snippets.append(c.get_text())
    elif (start_trigger_1==1) and ('AN-usecase-' in c.get_text()):
      start_trigger_1=0
      start_trigger_2=1
      snippets.append(c.get_text())
    elif (start_trigger_2==1):
      snippets.append(c.get_text())
    elif (start_trigger_2==0):
      snippets = []
    elif ('category' in c.get_text()):
      stop_trigger_1=1
      stop_trigger_2=0
      snippets.append(c.get_text())
    elif (stop_trigger_1==1) and ('reference' in c.get_text()):
      stop_trigger_1=0
      stop_trigger_2=1
      snippets.append(c.get_text())
    elif (stop_trigger_2==1):
      snippets.append(c.get_text())
      print(snippets)
      snippets = []
      start_trigger_1=0
      start_trigger_2=0
      stop_trigger_1=0
      stop_trigger_2=0
      print("************")
    elif (stop_trigger_2==0):
      snippets.append(c.get_text())


Streaming output truncated to the last 5000 lines.
● 
interoperable exchange of feedback or information from peers regarding the decisions and choices 
related to the AN behaviour.  

NOTE 1 – Peers may include humans and machines.  

NOTE 2 – Feedback may include exchange of information regarding the AN behaviour such as evolution, 
experimentation and adaptation. Contents of the information exchanged may include capabilities and status 
of AN components, e.g., knowledge base, orchestration and simulators. The format used for information 
exchange is for further study. 

Y series – Supplement 71 (07/2022) - prepublished version 

14

Page 17
7.3.2 Actor interactions and possible components 

Figure 5: Actor interactions for Peer-in-loop (including humans)

Figure 6: Possible components related to peer-in-loop (including humans) 

Actors involved (figure 5): 

Knowledge base: this actor stores knowledge which may be queried, stored and updated by various 
AN components. This may be imp

In [ ]:
from langchain.docstore.document import Document
cur_idx = -1
semantic_snippets = []
# Assumption: headings have higher font size than their respective content
for s in snippets:
    # if current snippet's font size > previous section's heading => it is a new heading
    if not semantic_snippets or s[1] > semantic_snippets[cur_idx].metadata['heading_font']:
        metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
        metadata.update(data.metadata)
        semantic_snippets.append(Document(page_content='',metadata=metadata))
        cur_idx += 1
        continue

    # if current snippet's font size <= previous section's content => content belongs to the same section (one can also create
    # a tree like structure for sub sections if needed but that may require some more thinking and may be data specific)
    if not semantic_snippets[cur_idx].metadata['content_font'] or s[1] <= semantic_snippets[cur_idx].metadata['content_font']:
        semantic_snippets[cur_idx].page_content += s[0]
        semantic_snippets[cur_idx].metadata['content_font'] = max(s[1], semantic_snippets[cur_idx].metadata['content_font'])
        continue

    # if current snippet's font size > previous section's content but less than previous section's heading than also make a new
    # section (e.g. title of a PDF will have the highest font size but we don't want it to subsume all sections)
    metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
    metadata.update(data.metadata)
    semantic_snippets.append(Document(page_content='',metadata=metadata))
    cur_idx += 1

In [ ]:
semantic_snippets[4]

Document(page_content='CAUTION ! \nPREPUBLISHED  RECOMMENDATION \nThis prepublication is an unedited version of a recently approved Recommendation. \nIt will be replaced by the published version after editing. Therefore, there will be \ndifferences between this prepublication and the published version.\nFOREWORD \nThe International Telecommunication Union (ITU) is the United Nations specialized agency in the field of \ntelecommunications, information and communication technologies (ICTs). The ITU Telecommunication \nStandardization Sector (ITU-T) is a permanent organ of ITU. ITU-T is responsible for studying technical, \noperating and tariff questions and issuing Recommendations on them with a view to standardizing \ntelecommunications on a worldwide basis. \nThe World Telecommunication Standardization Assembly (WTSA), which meets every four years, \nestablishes the topics for study by the ITU-T study groups which, in turn, produce Recommendations on \nthese topics. \nThe approval of I